### **Data Cleaning Workflow** 

---

#### **1. Import Data**  
- Drop records where:  
   - **`'sii'`** and **`'PCIAT_Total_Score'`** are `NaN`.  
- Save these as two different target variables:  
   - 🎯 **`sii`**  
   - 🎯 **`PCIAT`**

---

#### **2. Feature Cleaning**  
-  Drop all **features** that are present **only in the train set**.  
-  Remove features where **`NaN` values ≥ 60%** (Threshold: **`0.4`**).  
-  Merge all **FGC-Zone features**:  
   - Compute: **`feature + (zone_feature * 0.1)`**.

---

#### **3. Dataset Splitting**  
- **Split the dataset** into:  
   -  **Numerical Features**  
   -  **Categorical Features**

---

#### **4. Categorical Feature Encoding**  
- Use **1-Hot Encoding (OHE)**:  
   -  `pd.get_dummies()`

---

#### **5. Handle Outliers (Optional)**  
- 🚨 **[EVENTUALLY]**: Remove records with extremely **high values**.

---

#### **6. Missing Value Imputation**  
-  **Numerical Features**:  
   - Apply **Random Imputation**.  
   - Ensure the result matches the feature type:  
      - Truncate **float** to **integer** where needed.

---

#### **7. Model-Based Feature Engineering**  
-  Train a **classifier** (e.g., **kNN** or **SVC**) to compute **PCIAT_Total_Score**.  
   - If accuracy is **very high**:  
     - ➕ Add a **new feature** to the dataset filled by the model's predictions.

---

#### **8. Export Dataset**  
- **Export the cleaned dataset**.  
- Proceed to:  
   - Build **four classification models**.  
   - Perform **hyperparameter tuning**.  
   - Document appropriate **considerations**.
 


In [451]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

load_dotenv()
TRAIN_SET = os.getenv("TRAIN_PATH")
TEST_SET  = os.getenv("TEST_PATH")

train = pd.read_csv(TRAIN_SET)
test = pd.read_csv(TEST_SET)

#train = train.drop(['id'], axis=1)
#test = test.drop(['id'], axis=1)

In [452]:
train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [453]:
test.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,32.6909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,27.0552,NaN,NaN,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,45.9966,NaN,NaN,Winter,2.451,Summer,31.0,45.0,Winter,0.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,...,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [454]:
print("Train Shape -> ", train.shape, "\nTest Shape  -> ", test.shape)

Train Shape ->  (3960, 82) 
Test Shape  ->  (20, 59)


In [455]:
print("Info: ")
train.info()

Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3960 entries, 0 to 3959
Data columns (total 82 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      3960 non-null   object 
 1   Basic_Demos-Enroll_Season               3960 non-null   object 
 2   Basic_Demos-Age                         3960 non-null   int64  
 3   Basic_Demos-Sex                         3960 non-null   int64  
 4   CGAS-Season                             2555 non-null   object 
 5   CGAS-CGAS_Score                         2421 non-null   float64
 6   Physical-Season                         3310 non-null   object 
 7   Physical-BMI                            3022 non-null   float64
 8   Physical-Height                         3027 non-null   float64
 9   Physical-Weight                         3076 non-null   float64
 10  Physical-Waist_Circumference            898 non-null 

#### **2. Feature Cleaning**

In [456]:
# drop all the rows with missing values of the target variable

train = train.dropna(subset=['sii'])
SII_target = train['sii']
print("sii target   -> ", SII_target.shape)

sii target   ->  (2736,)


In [457]:
#drop train-only features

def intersect_features(train, test):
    sm_train = train[train.columns.intersection(test.columns)]
    return sm_train

In [458]:
train = intersect_features(train, test)
print("Train Shape   -> ", train.shape)

Train Shape   ->  (2736, 59)


In [459]:
#drop features with high percentage of missing values

def drop_columns(df, threshold):
    minimum_non_NaN = len(df) * threshold   
    dropped_columns = df.columns[df.isnull().sum() > (len(df) - minimum_non_NaN)].tolist()
    new_df = df.drop(columns=dropped_columns)
    
    return new_df, dropped_columns

In [460]:
X_train, dropped_features = drop_columns(train, 0.4)
print("Dropped Features are:", len(dropped_features), "   -> ", dropped_features)

Dropped Features are: 10    ->  ['Physical-Waist_Circumference', 'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec', 'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total']


In [461]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2736 entries, 0 to 3958
Data columns (total 49 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      2736 non-null   object 
 1   Basic_Demos-Enroll_Season               2736 non-null   object 
 2   Basic_Demos-Age                         2736 non-null   int64  
 3   Basic_Demos-Sex                         2736 non-null   int64  
 4   CGAS-Season                             2342 non-null   object 
 5   CGAS-CGAS_Score                         2342 non-null   float64
 6   Physical-Season                         2595 non-null   object 
 7   Physical-BMI                            2527 non-null   float64
 8   Physical-Height                         2530 non-null   float64
 9   Physical-Weight                         2572 non-null   float64
 10  Physical-Diastolic_BP                   2478 non-null   float64
 

In [462]:
"""
# merge fitness relevation features

def merge_fitness(df):
    df['Fitness_Endurance-Time'] = df['Fitness_Endurance-Time_Sec'] + (df['Fitness_Endurance-Time_Mins']*60) + df['Fitness_Endurance-Max_Stage']
    df = df.drop(['Fitness_Endurance-Time_Mins'], axis=1)
    df = df.drop(['Fitness_Endurance-Time_Sec'], axis=1)
    df = df.drop(['Fitness_Endurance-Max_Stage'], axis=1)
    return df
"""

"\n# merge fitness relevation features\n\ndef merge_fitness(df):\n    df['Fitness_Endurance-Time'] = df['Fitness_Endurance-Time_Sec'] + (df['Fitness_Endurance-Time_Mins']*60) + df['Fitness_Endurance-Max_Stage']\n    df = df.drop(['Fitness_Endurance-Time_Mins'], axis=1)\n    df = df.drop(['Fitness_Endurance-Time_Sec'], axis=1)\n    df = df.drop(['Fitness_Endurance-Max_Stage'], axis=1)\n    return df\n"

In [463]:
# Non sense because with the 0.4 threshold we already dropped the features. Uncomment if you want to merge the features once changed the threshold 

"""
X_merged_fitness = merge_fitness(X_train)
X_train = X_merged_fitness
"""

'\nX_merged_fitness = merge_fitness(X_train)\nX_train = X_merged_fitness\n'

In [464]:
# merge the FGC-Attr and the FGC-Attr_Zone features

def merge_fgc(train):
    FGC_features = [col for col in train.columns if 'FGC' in col]
    if 'FGC-Season' in FGC_features: 
        FGC_features.remove('FGC-Season')
    removed_features = 0
    zone_features_to_drop = []

    for feature in FGC_features:
        zone_feature = feature + '_Zone'

        if zone_feature in train.columns:
            print(f'Feature: {feature} - Zone: {zone_feature}')
            train[feature] = train[feature] + (train[zone_feature] * train[feature] * 0.2)
            zone_features_to_drop.append(zone_feature)
            removed_features += 1
    train = train.drop(zone_features_to_drop, axis=1)
    return train, removed_features

In [465]:
X_train, removed_features = merge_fgc(X_train)
print(f"Removed {removed_features} features from SII Train")

Feature: FGC-FGC_CU - Zone: FGC-FGC_CU_Zone
Feature: FGC-FGC_PU - Zone: FGC-FGC_PU_Zone
Feature: FGC-FGC_SRL - Zone: FGC-FGC_SRL_Zone
Feature: FGC-FGC_SRR - Zone: FGC-FGC_SRR_Zone
Feature: FGC-FGC_TL - Zone: FGC-FGC_TL_Zone
Removed 5 features from SII Train


In [466]:
X_train.head()

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,...,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_C-Season,PAQ_C-PAQ_C_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,...,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,NaN,Fall,3.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,...,39.4497,15.4107,27.0552,Fall,2.340,Fall,46.0,64.0,Summer,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,...,NaN,NaN,NaN,Summer,2.170,Fall,38.0,54.0,Summer,2.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,...,58.9338,26.4798,45.9966,Winter,2.451,Summer,31.0,45.0,Winter,0.0
5,001f3379,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,...,79.6982,35.3804,63.1265,Spring,4.110,Summer,40.0,56.0,Spring,0.0


In [467]:
X_train.shape

(2736, 44)

#### **3. Handle Outliers (Let's Try!)** 
I tried to filter outliers with the classic IQR approach. I used a relaxed version (with a multiplicative of 3 instead of 1.5) but from 2736 rows I obtain 550 records. I then try in this way:
I choose the most important features in my dataset (the ones starting with 'Physical', 'BIA' and 'FGC'). I choose a very large exclusive percentile (99.99) and I compute if a row does have at least a feature that contain a value higher than this 'threshold'. Given this high percentile I still remove a lot of records in my df: I do print histograms for every features and after applying this filter I do obtain a 1200 records dataframe.
I so decided to find out what are the more problematic features in my dataset and to only remove very extremely values

---

What do I select?

-> Basic-Demmos: Good distribution, do not select

-> CGAS: Only one feature, it has an acceptable distribution

-> Physical: Kinda good: longer tails but balanced

-> BIA_BMI: Kinda acceptable


---

-> FGC: PU and CU have a very long tail. Overperforming guys. I can delete that records. The other one have an acceptable distribution

-> BIA-BIA_BMC: It does have a single extremely high value. To be deleted only that specific one record

-> BIA_BMR: It does have a single extremely high value. To be deleted only that specific one record

-> BIA_DEE: It does have two extremely high value. To be deleted only that specific two records

-> BIA_ECW: It does have a single extremely high value. To be deleted only that specific one record

-> BIA_FFM: It does have a single extremely high value. To be deleted only that specific one record

-> BIA_FFMI: It does have a single extremely high value. To be deleted only that specific one record

-> BIA_FMI: It does have three extremely low value. To be deleted that three specific records

-> BIA_FAT: It does have three extremely low value. To be deleted that three specific records

-> BIA_ICW: It does have a single extremely high value. To be deleted only that specific one record

-> BIA_LDM: It does have two extremely high value. To be deleted only that specific two records

-> BIA_LST: It does have two extremely high value. To be deleted only that specific two records

-> BIA_SMM: It does have two extremely high value. To be deleted only that specific two records

-> BIA_TBW: It does have a single extremely high value. To be deleted only that specific one record


In [468]:
"""
to_remove = {
    'FGC-FGC_PU': 4,
    'FGC-FGC_CU': 4,
    'BIA-BIA_BMC': 1,
    'BIA-BIA_BMR': 1,
    'BIA-BIA_DEE': 2,
    'BIA-BIA_ECW': 1,
    'BIA-BIA_FFM': 1,
    'BIA-BIA_FFMI': 1,
    'BIA-BIA_FMI': 1,
    'BIA-BIA_ICW': 1,
    'BIA-BIA_LDM': 2,
    'BIA-BIA_LST': 2,
    'BIA-BIA_SMM': 2,
    'BIA-BIA_TBW': 1,
    'BIA-BIA_Fat': -3
}
"""

In [469]:
# TODO: ONLY ONE EXECUTION
# i want to delete all the rows which the features name is in the to_remove dictionary and I want to delete the exact number of rows specified in the dictionary

"""
def remove_extreme_records(df, prefix="BIA", max_removals=3):
    
    Removes at most a specified number of records with extreme values in a feature subset.
    
    Parameters:
    - df: DataFrame
    - prefix: Prefix of the feature subset to inspect (default is 'BIA').
    - max_removals: Maximum number of records to remove (default is 4).
    
    Returns:
    - Cleaned DataFrame with extreme records removed.
    
    # Filter the columns with the specified prefix
    subset_cols = [col for col in df.columns if col.startswith(prefix)]
    columns_to_remove = ['BIA-BIA_Activity_Level_num', 'BIA-Season']

    for col in columns_to_remove:
        if col in subset_cols:
            subset_cols.remove(col)

    print(f"Subset Columns: {subset_cols}")
    
    # Identify extreme values across all subset features
    extreme_indices = []
    for col in subset_cols:
        # Find absolute extreme values in the column
        extremes = df[col].abs().sort_values(ascending=False).head(max_removals)
        print(f"Feature '{col}': Extreme Values -> {extremes.tolist()}")
        extreme_indices.extend(extremes.index.tolist())
    
    # Deduplicate indices and limit to max_removals
    unique_extreme_indices = list(set(extreme_indices))
    unique_extreme_indices = unique_extreme_indices[:max_removals]
    print(f"Final Rows to Remove (Limited to {max_removals}): {unique_extreme_indices}")
    
    # Drop the rows
    cleaned_df = df.drop(index=unique_extreme_indices)
    return cleaned_df
"""

In [470]:
"""
X_train_no_outliers = remove_extreme_records(X_train, "BIA", 3)
print("Train Shape -> ", X_train_no_outliers.shape)
"""

Subset Columns: ['BIA-BIA_BMC', 'BIA-BIA_BMI', 'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM', 'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num', 'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM', 'BIA-BIA_TBW']
Feature 'BIA-BIA_BMC': Extreme Values -> [4115.36, 401.002, 29.463]
Feature 'BIA-BIA_BMI': Extreme Values -> [48.3754, 46.1079, 45.311]
Feature 'BIA-BIA_BMR': Extreme Values -> [83152.2, 11540.8, 3987.68]
Feature 'BIA-BIA_DEE': Extreme Values -> [124728.0, 17311.2, 7994.08]
Feature 'BIA-BIA_ECW': Extreme Values -> [3233.0, 350.849, 115.285]
Feature 'BIA-BIA_FFM': Extreme Values -> [8799.08, 1171.51, 367.004]
Feature 'BIA-BIA_FFMI': Extreme Values -> [217.771, 82.4902, 65.5384]
Feature 'BIA-BIA_FMI': Extreme Values -> [194.163, 66.378, 45.8722]
Feature 'BIA-BIA_Fat': Extreme Values -> [8745.08, 1044.51, 217.522]
Feature 'BIA-BIA_Frame_num': Extreme Values -> [3.0, 3.0, 3.0]
Feature 'BIA-BIA_ICW': Extreme Values -> [2457.91, 428.264, 152.738]


### **La rimozione degli outlier procede a mano per impossibilità nell'agire in modo convenzionale con gli outliers**

#### **. Dataset Splitting**